In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from Bio import SeqIO

## get genome size from gbk file

In [2]:
infile = SeqIO.read('/home/m/genome_files/REF-seq_CLJU.gbk','gb')
genome_size=''
for r in infile.features[0:1]:
    genome_size= len(r)
print genome_size

4630065


## import omics df which contains all data

In [3]:
omics= pd.read_excel('/home/m/Dropbox/Clostridium_paper/CLJU_omics_df.xls')

In [4]:
omics.head()

,gene_id,old_gene_id,av_ribo_rpkm_co,av_ribo_rpkm_hf,av_ribo_rpkm_h2c,av_rna_rpkm_co,av_rna_rpkm_hf,av_rna_rpkm_h2c,co_te,hf_te,h2c_te
0,CLJU_RS00005,CLJU_c00010,30.907470,54.283921,56.255488,302.166750,301.233050,446.001139,0.102286,0.180206,0.126133
1,CLJU_RS00010,CLJU_c00020,5.999725,33.563496,5.768107,25.879322,35.111517,31.742726,0.231835,0.955911,0.181714
2,CLJU_RS00015,CLJU_c00030,7.930030,20.794130,18.230279,64.220289,85.874412,64.114905,0.123482,0.242146,0.284338
3,CLJU_RS00020,CLJU_c00040,5.884056,10.831297,5.337636,109.500968,185.040620,82.691535,0.053735,0.058535,0.064549
4,CLJU_RS00025,CLJU_c00050,12.337119,26.791583,18.526946,132.895890,269.184048,104.904829,0.092833,0.099529,0.176607


## Omics data lacks gene coordinates, will get them from gbk file

In [5]:
gene_id=[]
start=[]
stop=[]
strand=[]
for i in infile.features:
    if i.type == 'CDS':
        gene_id.append( i.qualifiers['locus_tag'][0])
        start.append( i.location.start+1)
        stop.append(i.location.end)
        if i.strand == -1:
            strand.append('-')
        elif i.strand== 1:
            strand.append('+')
df = pd.DataFrame({'gene_id':gene_id, 'start':start, 'stop': stop, 'strand':strand},
                  columns=['gene_id','start','stop','strand'])

In [6]:
omics = pd.merge(omics, df, on='gene_id')

In [7]:
omics=omics.replace(['inf', '-inf'], 0.0)

In [8]:
#generate arrays of zeros the size of genome
te_co_pos=np.zeros(genome_size)
te_hf_pos=np.zeros(genome_size)
te_h2c_pos=np.zeros(genome_size)
te_co_neg=np.zeros(genome_size)
te_hf_neg=np.zeros(genome_size)
te_h2c_neg=np.zeros(genome_size)
#modify each array with te values from co, hf, and h2c, otherwise it's zero
for idx,r in omics.iterrows():
    st = r['start']
    sp = r['stop']
    co = r['co_te']
    hf = r['hf_te']
    h2c= r['h2c_te']
    sd = r['strand']
    if sd =='+':
        #te_co[st:sp]=[co for i in range(st,sp)]
        te_co_pos[st:sp]=co
        te_hf_pos[st:sp]=hf
        te_h2c_pos[st:sp]=h2c
    elif sd=='-':
        #te_co[st:sp]=[-co for i in range(st,sp)]
        te_co_neg[st:sp]=-co
        te_hf_neg[st:sp]=-hf
        te_h2c_neg[st:sp]=-h2c

#wigs_pos = pd.DataFrame({'co_te':te_co_pos, 'hf_te':te_hf_pos, 'h2c_te':te_h2c_pos},
#                   columns=['co_te', 'hf_te', 'h2c_te'])
#wigs_neg=pd.DataFrame({'co_te':te_co_neg, 'hf_te':te_hf_neg, 'h2c_te':te_h2c_neg},
#                   columns=['co_te', 'hf_te', 'h2c_te'])

## Generating wig files

In [10]:
headerco='tracktype=wiggle_0\tname=te_co_pos\tgraphType=points\tvisibility=full\tcolor=168,130,88\nfixedStep\tchrom=NC_014328\tstart=1\tstep=1\tspan=1'
headerhf='track   type=wiggle_0   name=te_hf_pos      graphType=points        visibility=full color=168,130,88\nfixedStep       chrom=NC_014328 start=1 step=1  span=1'
headerh2c='track   type=wiggle_0   name=te_h2c_pos      graphType=points        visibility=full color=168,130,88\nfixedStep       chrom=NC_014328 start=1 step=1  span=1'

In [11]:
headercong='tracktype=wiggle_0\tname=te_co_neg\tgraphType=points\tvisibility=full\tcolor=168,130,88\nfixedStep\tchrom=NC_014328\tstart=1\tstep=1\tspan=1'
headerhfng='track   type=wiggle_0   name=te_hf_neg      graphType=points        visibility=full color=168,130,88\nfixedStep       chrom=NC_014328 start=1 step=1  span=1'
headerh2cng='track   type=wiggle_0   name=te_h2c_neg      graphType=points        visibility=full color=168,130,88\nfixedStep       chrom=NC_014328 start=1 step=1  span=1'

In [22]:
te_h2c_neg=np.nan_to_num(te_h2c_neg)
te_h2c_pos=np.nan_to_num(te_h2c_pos)

In [23]:
np.savetxt('/home/m/Dropbox/Clostridium_paper/wig_files/co_te_pos.wig',te_co_pos,
           fmt='%.1f', header=headerco, comments='',newline='\n', delimiter='')
np.savetxt('/home/m/Dropbox/Clostridium_paper/wig_files/hf_te_pos.wig',te_hf_pos,
           fmt='%.1f', header=headerhf, comments='',newline='\n')
np.savetxt('/home/m/Dropbox/Clostridium_paper/wig_files/h2c_te_pos.wig',te_h2c_pos,
           fmt='%.1f', header=headerh2c, comments='',newline='\n')

In [24]:
np.savetxt('/home/m/Dropbox/Clostridium_paper/wig_files/co_te_neg.wig',te_co_neg,
           fmt='%.1f', header=headercong, comments='',newline='\n', delimiter='')
np.savetxt('/home/m/Dropbox/Clostridium_paper/wig_files/hf_te_neg.wig',te_hf_neg,
           fmt='%.1f', header=headerhfng, comments='',newline='\n')
np.savetxt('/home/m/Dropbox/Clostridium_paper/wig_files/h2c_te_neg.wig',te_h2c_neg,
           fmt='%.1f', header=headerh2cng, comments='',newline='\n')